In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.5.3
2.8.4


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [7]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.1_Dike Investment Costs'), function=<function sum_over at 0x00000138969F49D0>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x00000138969F49D0>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage', 'A.2_Dike Investment Costs'), function=<function sum_over at 0x00000138969F49D0>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x00000138969F49D0>)
ScalarOutcome('A.3 Total Costs', variable_name=('A.3_Expected Annual Damage', 'A.3_Dike Investment Costs'), function=<function sum_over at 0x00000138969F49D0>)
ScalarOutcome('A.3_Expected Number of Deaths', variable_name=('A.3_Expected Number of Deaths',), function=<function sum_over at 0x00000138969F49D0>)
ScalarOutcome('A.4 Total Costs', variable_name=('A.4_Expected Annual Dama

In [8]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [00:20<00:00,  9.62it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [9]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,123,278.653186,1.0,0.900730,82.301184,1.0,0.469242,206.299705,10.0,0.543352,...,4,4,2,3,8,9,2,4,0,dikesnet
1,34,227.590809,1.5,0.690532,79.598372,1.5,0.692121,282.448400,1.0,0.926301,...,4,4,2,3,8,9,2,5,0,dikesnet
2,18,98.922545,1.5,0.474723,143.869464,1.5,0.317218,68.939125,1.5,0.791970,...,4,4,2,3,8,9,2,6,0,dikesnet
3,28,212.620573,1.5,0.751640,337.730117,1.0,0.120435,184.480858,1.0,0.561284,...,4,4,2,3,8,9,2,7,0,dikesnet
4,67,262.271149,1.0,0.508797,35.348007,1.0,0.987752,179.637899,1.0,0.982199,...,4,4,2,3,8,9,2,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1,122.253406,10.0,0.029308,122.489681,10.0,0.207944,144.243943,1.5,0.830845,...,8,5,0,1,9,3,1,49,3,dikesnet
196,19,326.985291,1.0,0.006847,272.570952,1.5,0.076060,112.428788,1.0,0.535829,...,8,5,0,1,9,3,1,50,3,dikesnet
197,110,145.423053,1.0,0.252276,189.254943,10.0,0.602614,317.987958,10.0,0.036225,...,8,5,0,1,9,3,1,51,3,dikesnet
198,118,63.281322,1.5,0.157931,136.107246,10.0,0.540119,47.748254,1.5,0.674464,...,8,5,0,1,9,3,1,52,3,dikesnet


In [10]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,2.810092e+08,0.000000,2.007270e+08,0.002720,8.358240e+07,0.000000,3.037829e+07,0.0,1.608193e+08,0.000000,1.134000e+09,843.767941
1,2.810092e+08,0.000000,1.954977e+08,0.001085,8.358240e+07,0.000000,3.037829e+07,0.0,1.608193e+08,0.000000,1.134000e+09,325.644278
2,2.810092e+08,0.000000,2.087849e+08,0.004306,8.358240e+07,0.000000,3.037829e+07,0.0,1.608193e+08,0.000000,1.134000e+09,1382.468212
3,2.810092e+08,0.000000,3.474838e+08,0.015717,8.358240e+07,0.000000,3.037829e+07,0.0,1.608193e+08,0.000000,1.134000e+09,5445.758663
4,2.810092e+08,0.000000,1.881975e+08,0.000088,8.358240e+07,0.000000,3.037829e+07,0.0,1.964852e+08,0.007046,1.134000e+09,2827.640094
...,...,...,...,...,...,...,...,...,...,...,...,...
195,6.369542e+08,0.185389,1.007594e+08,0.000000,6.231890e+07,0.000000,3.059122e+07,0.0,1.224382e+08,0.000000,7.914000e+08,4708.007538
196,1.229452e+09,0.197407,1.007594e+08,0.000000,6.231890e+07,0.000000,3.059122e+07,0.0,1.224382e+08,0.000000,7.914000e+08,4982.978312
197,9.904047e+07,0.002476,1.007594e+08,0.000000,8.992644e+07,0.017060,3.059122e+07,0.0,3.485132e+08,0.070003,7.914000e+08,7628.319040
198,1.317854e+08,0.009704,1.007594e+08,0.000000,6.231890e+07,0.000000,3.059122e+07,0.0,1.322618e+08,0.003183,7.914000e+08,522.048675


In [45]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "Policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "Policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "Policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "Policy 0",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 0, "2_RfR 1": 0, "3_RfR 2": 0, "A.3_DikeIncrease 0": 0}
        )
    )
    ,
    Policy(
        "Policy DImax",
        **dict(
            get_do_nothing_dict(),
            **{
               "A.1_DikeIncrease 0": 5,"A.2_DikeIncrease 0": 5,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 5, "A.5_DikeIncrease 0": 5}
        )
    )
    ,
        Policy(
        "Policy RfRmax",
        **dict(
            get_do_nothing_dict(),
            **{
               "0_RfR 0": 1, "1_RfR 0": 1, "2_RfR 0": 1, "3_RfR 0": 1, "4_RfR 0": 1, "5_RfR 0": 1}
        )
    )
]

In [46]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 6 policies * 1 model(s) = 600 experiments
100%|████████████████████████████████████████| 600/600 [06:22<00:00,  1.57it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [47]:
experiments, outcomes = results

In [16]:
# only works because we have scalar outcomes
pd.DataFrame(experiments)

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model
0,107,344.093200,10.0,0.111224,89.862928,1.5,0.634971,196.052315,10.0,0.390858,...,0,0,0,0,0,0,0,54,policy 1,dikesnet
1,84,324.971901,1.5,0.996298,60.499543,1.0,0.044678,87.134599,10.0,0.532627,...,0,0,0,0,0,0,0,55,policy 1,dikesnet
2,63,220.954049,10.0,0.299327,269.832366,1.5,0.877959,273.924654,1.5,0.956891,...,0,0,0,0,0,0,0,56,policy 1,dikesnet
3,76,199.477711,10.0,0.360714,282.875524,1.5,0.202087,131.533247,1.5,0.417844,...,0,0,0,0,0,0,0,57,policy 1,dikesnet
4,83,139.072508,1.0,0.024760,62.711536,10.0,0.582257,229.168199,1.0,0.239274,...,0,0,0,0,0,0,0,58,policy 1,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,47,256.165509,1.0,0.427065,214.357210,1.0,0.967019,41.850430,10.0,0.888713,...,0,0,0,0,0,0,0,149,policy 3,dikesnet
296,126,60.634169,10.0,0.267810,337.518045,10.0,0.212783,124.575608,10.0,0.428133,...,0,0,0,0,0,0,0,150,policy 3,dikesnet
297,103,298.503108,1.0,0.053448,176.525467,1.0,0.154275,237.770913,1.0,0.463362,...,0,0,0,0,0,0,0,151,policy 3,dikesnet
298,71,247.374858,1.0,0.442643,172.487767,1.5,0.006238,307.829366,10.0,0.480555,...,0,0,0,0,0,0,0,152,policy 3,dikesnet


In [49]:
from funs_viz import tidy_results

results_df = tidy_results(results)
results_df.to_csv("results/test_1.csv")

results_df

,A.0_ID flood wave shape,discount rate 0,discount rate 1,discount rate 2,EWS_DaysToThreat,scenario,policy,model,RfR Total Costs,Expected Evacuation Costs,...,Brate,pfail,RfR 0,RfR 1,RfR 2,DikeIncrease 0,DikeIncrease 1,DikeIncrease 2,Total costs,Expected Number of Deaths
0,92,2.5,3.5,1.5,0,654,Policy 1,dikesnet,253800000.0,0.0,...,1.5,0.436880,1,1,1,5,0,0,5.397251e+07,0.000000
1,22,1.5,1.5,4.5,0,655,Policy 1,dikesnet,253800000.0,0.0,...,1.0,0.324090,1,1,1,5,0,0,5.397251e+07,0.000000
2,113,2.5,2.5,4.5,0,656,Policy 1,dikesnet,253800000.0,0.0,...,1.5,0.986039,1,1,1,5,0,0,5.397251e+07,0.000000
3,2,2.5,4.5,4.5,0,657,Policy 1,dikesnet,253800000.0,0.0,...,1.0,0.650638,1,1,1,5,0,0,5.397251e+07,0.000000
4,116,2.5,2.5,3.5,0,658,Policy 1,dikesnet,253800000.0,0.0,...,1.5,0.000089,1,1,1,5,0,0,3.664537e+08,0.205471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,40,1.5,1.5,1.5,0,749,Policy RfRmax,dikesnet,710400000.0,0.0,...,1.5,0.791340,1,0,0,0,0,0,0.000000e+00,0.000000
2996,55,4.5,4.5,4.5,0,750,Policy RfRmax,dikesnet,710400000.0,0.0,...,1.5,0.170816,1,0,0,0,0,0,0.000000e+00,0.000000
2997,1,4.5,3.5,1.5,0,751,Policy RfRmax,dikesnet,710400000.0,0.0,...,10.0,0.775817,1,0,0,0,0,0,0.000000e+00,0.000000
2998,115,2.5,1.5,3.5,0,752,Policy RfRmax,dikesnet,710400000.0,0.0,...,1.0,0.235980,1,0,0,0,0,0,0.000000e+00,0.000000


In [23]:
results_df.to_csv("results/test_2.csv")